In [1]:
#!/usr/bin/env python3

In [2]:
# import librarys
import pandas as pd
import subprocess
import os
import sys
import json
from urllib.parse import urlsplit
import shutil

In [89]:
# pathes
csv = '../data/database/db_02_updated.csv'
git_store = '../data/repositories/git/'
others_store = '../data/repositories/others/'

# pathes for blobcity
csv = '../data/database/blobcity_01_index.csv'
git_store = '../data/repositories/blobcity/git/'

In [5]:
# read csv data
df = pd.read_csv(csv, sep=';')

# print shape
print(df.shape)

(1012, 7)


In [6]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [62]:
# download function for git-repositories
def git_clone(url, index, extra, silent=False):
    # return if link is empty
    if isinstance(url, str):
        # base path
        #path = git_store+str(index).zfill(4)+'_'

        # split url
        split_url = urlsplit(url)
        
        # url formatting
        #split_url = split_url.netloc.replace('www.', '')
        
        if not silent:
            print(url)
        
        # check if url is a git repository
        if split_url.netloc == 'github.com':
            
            author = split_url.path.split('/', 2)[1]
            git = split_url.path.split('/', 3)[2]
            git = git.replace('%20', ' ')
            
            if not silent:
                print(author, git)

            # base path
            #path = git_store+author+git
            
            # url formatting
            url = url.replace('www.', '')
            url = url.replace('%20', ' ')
            url = url.split("blob",1)[0]
            url = url.split("tree",1)[0]

            # project path
            name = os.path.basename(os.path.normpath(url))
            path_project = os.path.join(git_store, author, git)
            
            if not silent:
                print(path_project)

            # check if folder exists
            project_exists = os.path.isdir(path_project)
            
            # meta path
            path_meta = os.path.join(git_store, author, git, '.meta/') + 'log_clone.txt'
            
            # download if folder does not exist
            if not project_exists:
                if not silent:
                    print('---')
                    print(path_project)

                # git clone
                try:
                    output = subprocess.getoutput("git clone {} {}".format(url, path_project))
                    if not silent:
                        print(output)
                    
                    # create meta file for output
                    create_folder(path_meta)
                    with open(path_meta, 'w') as file:
                        file.write(output)
                except Exception as e:
                    if silent:
                        print(path_project)
                    print("Oops!", e.__class__, "occurred.")
                    print(e)

                
                    
            # check if additional_info.json exist
            path = os.path.join(git_store, author, git, '.meta/') + 'additional_info.json'
            if not silent:
                print(path)
            if not os.path.exists(path):
                if not silent:
                    print('json not found')
                try:
                    create_folder(path)
                    with open(path, 'w') as file:
                        file.write(json.dumps(extra))
                except Exception as e:
                    if silent:
                        print(path)
                        print('json not found')
                    print("Oops!", e.__class__, "occurred.")
                    print(e)
                    
    else:
        print('empty url on row '+str(index))

In [90]:
# download all git repositories
for index, row in df.iterrows():
    #url = row['Link']
    
    # blobcity
    url = row['github_link']
    
    '''
    #Industry	Type	Name	Link	Description	Added to Github
    extra = {
        'industry': row['Industry'],
        'type': row['Type'],
        'name': row['Name'],
        'description': row['Description'],
    }
    '''
    
    # blobcity
    #title	blobcity_link	description	github_link	tags	created_at	modified_at
    extra = {
        'title': row['title'],
        'blobcity_link': row['blobcity_link'],
        'description': row['description'],
        'tags': row['tags'],
        'created_at': row['created_at'],
        'modified_at': row['modified_at'],
    }

    #print(url)
    # sort out 404-urls
    if not isinstance(url, float) and not 'sfbrigade' in url:
        git_clone(url, index, extra, silent=True)

In [91]:
# list folders with missing '.git' to provide fixing

# trashing requires admin or user priviliges
trash = False
trash_bin = trash_store = '../data/repositories/zzz/'

for folder in os.listdir(git_store):
    for subfolder in os.listdir(os.path.join(git_store, folder)):
    #subfolder = os.listdir(os.path.join(git_store, folder))[0]
        
        path = os.path.join(git_store, folder, subfolder, '.git')
        if not os.path.isdir(path):
            print ('git not found:', path)
            if trash:
                path = os.path.join(git_store, folder, subfolder)
                trash_path = os.path.join(trash_bin, folder, subfolder)
                shutil.move(path, trash_path)
                break
           
        path = os.path.join(git_store, folder, subfolder, '.meta')+'/log_clone.txt'
        if not os.path.exists(path):
            print ('log not found:', path)
            if trash:
                path = os.path.join(git_store, folder, subfolder)
                trash_path = os.path.join(trash_bin, folder, subfolder)
                shutil.move(path, trash_path)
                break
           
        path = os.path.join(git_store, folder, subfolder, '.meta')+'/additional_info.json'
        if not os.path.exists(path):
            print ('info not found:', path)

log not found: ../data/repositories/blobcity/git/edvardHua\Articles\.meta/log_clone.txt
log not found: ../data/repositories/blobcity/git/FourSpaces\notebook\.meta/log_clone.txt
log not found: ../data/repositories/blobcity/git/Holy-Shine\Pytorch-notebook\.meta/log_clone.txt
git not found: ../data/repositories/blobcity/git/hortonworks-gallery\ambari-zeppelin-service\.git
git not found: ../data/repositories/blobcity/git/jupyter-attic\jupyter-js-notebook\.git
log not found: ../data/repositories/blobcity/git/jupyter-on-openshift\jupyter-notebooks\.meta/log_clone.txt
log not found: ../data/repositories/blobcity/git/kaleko\CourseraML\.meta/log_clone.txt
log not found: ../data/repositories/blobcity/git/Kennytian\learning-react-native\.meta/log_clone.txt
git not found: ../data/repositories/blobcity/git/ketch\teaching-numerics-with-notebooks\.git
log not found: ../data/repositories/blobcity/git/kokes\nbviewer.js\.meta/log_clone.txt
log not found: ../data/repositories/blobcity/git/plotly\IPython-